In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Datos de ejemplo
steps = 100  # Número de pasos de tiempo en cada serie temporal

data1 = {
    'humidity': np.random.rand(steps),
    'temperature': np.random.rand(steps) * 30,
    'wind_speed': np.random.rand(steps) * 10,
    'precipitation': np.sin(np.linspace(0, 100, steps))
}
data2 = {
    'humidity': np.random.rand(steps),
    'temperature': np.random.rand(steps) * 30,
    'wind_speed': np.random.rand(steps) * 10,
    'precipitation': np.sin(np.linspace(0, 100, steps))
}
data3 = {
    'humidity': np.random.rand(steps),
    'temperature': np.random.rand(steps) * 30,
    'wind_speed': np.random.rand(steps) * 10,
    'precipitation': np.sin(np.linspace(0, 100, steps))
}

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)
df3 = pd.DataFrame(data3)

# Normalizar los datos
scaler = MinMaxScaler()
data1_scaled = scaler.fit_transform(df1)
data2_scaled = scaler.fit_transform(df2)
data3_scaled = scaler.fit_transform(df3)

# Crear secuencias
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

SEQ_LENGTH = 10  # Longitud de la secuencia
X1, y1 = create_sequences(data1_scaled, SEQ_LENGTH)
X2, y2 = create_sequences(data2_scaled, SEQ_LENGTH)
X3, y3 = create_sequences(data3_scaled, SEQ_LENGTH)

# Concatenar las secuencias de las tres series temporales
X = np.concatenate((X1, X2, X3), axis=0)
y = np.concatenate((y1, y2, y3), axis=0)

# Remodelar los datos para ConvLSTM2D (samples, time_steps, rows, cols, channels)
X = X.reshape((X.shape[0], X.shape[1], 1, 1, X.shape[2]))

# Dividir en conjuntos de entrenamiento y prueba
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Construir el modelo ConvLSTM
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1, 1), activation='relu',
                     input_shape=(SEQ_LENGTH, 1, 1, X.shape[4]), return_sequences=True))
model.add(BatchNormalization())
model.add(ConvLSTM2D(filters=64, kernel_size=(1, 1), activation='relu', return_sequences=False))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(X.shape[4]))

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Evaluar el modelo
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# Hacer predicciones
predictions = model.predict(X_test)


Epoch 1/20
6/6 [==============================] - 19s 3s/step - loss: 1.1579 - val_loss: 0.2918
Epoch 2/20
6/6 [==============================] - 11s 2s/step - loss: 0.5196 - val_loss: 0.2617
Epoch 3/20
6/6 [==============================] - 16s 3s/step - loss: 0.3727 - val_loss: 0.2335
Epoch 4/20
6/6 [==============================] - 16s 3s/step - loss: 0.2678 - val_loss: 0.2058
Epoch 5/20
6/6 [==============================] - 14s 2s/step - loss: 0.2290 - val_loss: 0.1860
Epoch 6/20
6/6 [==============================] - 16s 3s/step - loss: 0.1836 - val_loss: 0.1699
Epoch 7/20
6/6 [==============================] - 14s 2s/step - loss: 0.1608 - val_loss: 0.1543
Epoch 8/20
6/6 [==============================] - 14s 2s/step - loss: 0.1352 - val_loss: 0.1414
Epoch 9/20
6/6 [==============================] - 17s 3s/step - loss: 0.1127 - val_loss: 0.1326
Epoch 10/20
6/6 [==============================] - 16s 3s/step - loss: 0.1012 - val_loss: 0.1249
Epoch 11/20
6/6 [======================

In [8]:
X

array([[[[[2.70041349e-01, 2.75951930e-01, 5.42758426e-01,
           4.99981961e-01]]],


        [[[6.88229921e-01, 9.10185459e-01, 3.48333770e-01,
           9.23443839e-01]]],


        [[[3.28658151e-01, 9.56598369e-01, 5.19722061e-01,
           9.50354994e-01]]],


        ...,


        [[[1.71834741e-01, 7.55544685e-01, 2.91489270e-01,
           8.54301338e-01]]],


        [[[4.87512543e-01, 2.78779582e-01, 5.50264394e-01,
           9.87196427e-01]]],


        [[[8.71545806e-01, 1.46872608e-01, 4.10523370e-01,
           6.63839705e-01]]]],



       [[[[6.88229921e-01, 9.10185459e-01, 3.48333770e-01,
           9.23443839e-01]]],


        [[[3.28658151e-01, 9.56598369e-01, 5.19722061e-01,
           9.50354994e-01]]],


        [[[7.35118859e-01, 1.05290395e-01, 1.53584746e-01,
           5.55514484e-01]]],


        ...,


        [[[4.87512543e-01, 2.78779582e-01, 5.50264394e-01,
           9.87196427e-01]]],


        [[[8.71545806e-01, 1.46872608e-01, 4.10523370e-01,